# Structured Query Language (SQL)

### RECAP:

1. Pandas Apply
2. Pandas Groupby
3. Pandas Datatypes
4. Pandas Cleaning Strategies

#### SQL in python for data scientists

In this session we'll be using sqlite. In industries, most companies use mysql or postgresql (cloud solutions). The language and logic is similar.

SQL is a language designed to work with data

sqlite, mysql, etc are database systems that store data in a relational way

<img src="./images/sql.jpg" height=400, width=400>

With a lot more tables:

<img src="images/relational.png" height=400 width=400>

We use SQL because

1. It's an efficient data querying language
2. Industry standard
3. Scalable database system

for example: to find a row in a CSV file you need parse line by line, but in SQL it's index based so the look up time is exponentially quicker.

In [1]:
import pandas as pd
import sqlite3

### Types of SQL query

1. DQL (Data Querying Language): Anything that reads data without permanently changing the database itself
2. DML (Data Manipulating Language): Anything that changes the data inside of tables and rows, but does not change internal structure like column names or column data types
3. DDL (Data Definition language): Anything that changes the structure of databases, such as the columns. This is also used to create new tables and remove tables in schemas.
4. DCL (Data Control Language): Acts as the authentication system to ensure the correct people are given the correct permissions to access correct databases
5. TCL (Transaction Control Language): A system to ensure robustness of manipulations in real production

As a data scientist, we'll mostly use SQL to extract information from a database, and manipulate them through pandas. This is the standard workflow. So this session will only focus on DQL

The basic syntax of DQL:

```{sql}
SELECT Columns, Expressions
FROM Tables, Joins
WHERE Condition
GROUP BY Column
HAVING Condition
ORDER BY Column
LIMIT number OFFSET number;
```

All of this is considered one query, you can do query to query operations like set operations, subqueries, CTEs, etc. But we won't cover that in this session

SQL parses this differently:

1. From: gathers the tables to create a single virtual table to query from
2. Where: initial filtering of rows
3. Group By: changes structure of virtual table
4. Having: filtering of rows based on group by
5. Select: filtering of columns and expressions on columns
6. Order By: sorting
7. Limit / Offset: controls the output

Structure of today's session

1. Topic 1: Select, From, Where (50 mins)
2. Topic 2: Group by, Having, Order By, Limit / Offset (50 mins)

In [2]:
# setup a connection with the sqlite file
conn = sqlite3.connect("./data/chinook.sqlite")

# retrieves the cursor (allows us to write SQL)
cursor = conn.cursor()

# Executes the script to read table info in the file
cursor.execute("PRAGMA table_list;") 

# fetches result
result = cursor.fetchall() 

for table in result:
    print(f"table: {table[1]}")

table: Track
table: PlaylistTrack
table: Playlist
table: sqlite_schema
table: Artist
table: Customer
table: Employee
table: Genre
table: Invoice
table: Album
table: InvoiceLine
table: MediaType
table: sqlite_temp_schema


In [3]:
cursor.execute("SELECT * FROM Track LIMIT 2;")

result = cursor.fetchall()

result

[(1,
  'For Those About To Rock (We Salute You)',
  1,
  1,
  1,
  'Angus Young, Malcolm Young, Brian Johnson',
  343719,
  11170334,
  0.99),
 (2, 'Balls to the Wall', 2, 2, 1, None, 342562, 5510424, 0.99)]

Output looks messy, but we can actually use pandas to turn this into a dataframe!

In [4]:
df = pd.read_sql("SELECT * FROM Track", conn) 
df.head(2)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


### SQL: From

for the first one, we'll look at the FROM argument. FROM lets you gather multiple tables and create a virtual table to do the rest of your queries on.

The basic syntax is:

```{sql}
FROM table1 t1 INNER JOIN table2 t2 ON t1.id = t2.id
```

But you can even do more complicated nestings like:

```{sql}
FROM table1 t1 INNER JOIN table2 t2 ON t1.id = t2.id LEFT JOIN table3 t3 ON t2.name = t3.name
```

In [30]:
# Example

s1 = "Track t"
s2 = "Album a ON t.AlbumId = a.AlbumId"
s3 = "Artist b ON a.ArtistId = b.ArtistId"

df = pd.read_sql("SELECT t.Name, a.title, b.Name " \
f"FROM {s1} INNER JOIN {s2} INNER JOIN {s3}", conn) 

df

,Name,Title,Name
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC
1,Balls to the Wall,Balls to the Wall,Accept
2,Fast As a Shark,Restless and Wild,Accept
3,Restless and Wild,Restless and Wild,Accept
4,Princess of the Dawn,Restless and Wild,Accept
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome,Eugene Ormandy
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,Emerson String Quartet
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music,Nash Ensemble


### SQL: SELECT

Next, we'll look at the SELECT argument. Select tells you how you want your output to look like before sorting, row filtering and offsetting.

The basic syntax is:

```{sql}
SELECT [DISTINCT] col, Expression
```

Here is DISTINCT is optional, if you add it you'll be getting distinct rows like pandas unique function

Expression here can be a lot of different types but the main ones are:

1. Arithmetic: col1 * col2
2. Aggregation: function(col1) -- collapses into one row, only for groupby
3. Cases: CASE WHEN condition THEN result ELSE result END AS col1
4. Window Functions: function(col1) OVER (PARTITION BY col ORDER BY col)

We won't get too much into window functions, and we will cover aggregation in group by



In [37]:
# Examples

s1 = "name"
s2 = "Bytes / Milliseconds AS byte_milli_ratio"
s3 = "CASE WHEN UnitPrice = 0 THEN 'free'" \
"WHEN UnitPrice > 1 THEN 'expensive' ELSE 'cheap'" \
"END AS cost_summary" 

df = pd.read_sql(f"SELECT {s1}, {s2}, {s3} FROM Track", conn) 

df

,Name,byte_milli_ratio,cost_summary
0,For Those About To Rock (We Salute You),32,cheap
1,Balls to the Wall,16,cheap
2,Fast As a Shark,17,cheap
3,Restless and Wild,17,cheap
4,Princess of the Dawn,16,cheap
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,16,cheap
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",16,cheap
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",17,cheap
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",16,cheap


### SQL: WHERE

Where is essentially an initial filtering (like boolean indexxing in pandas) right after FROM, so it's the first ever filter before select, group by, etc.

The syntax is simply:

```{sql}
WHERE condition
```

Here condition could be: col1 > 5 or col2 = "tree", etc

In [35]:
# Example:

df = pd.read_sql(f"SELECT name, UnitPrice FROM Track WHERE UnitPrice > 1", conn) 

df

,Name,UnitPrice
0,Battlestar Galactica: The Story So Far,1.99
1,Occupation / Precipice,1.99
2,"Exodus, Pt. 1",1.99
3,"Exodus, Pt. 2",1.99
4,Collaborators,1.99
...,...,...
208,"There's No Place Like Home, Pt. 1",1.99
209,"There's No Place Like Home, Pt. 2",1.99
210,"There's No Place Like Home, Pt. 3",1.99
211,Branch Closing,1.99
